In [1]:
# this notebook is to utilised transfer learning to train a model on a dataset to classify medical image data on breast cancer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import pandas as pd
import torchvision
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import os
import skimage.io as io
import time
import copy

In [2]:
# get root directory
root_dir = '/media/victoru/B612CEC512CE8A37/ai50/pytorch_test/data/breatcancer_data/archive'
id_dir = os.listdir(root_dir)
label_dir = os.listdir(os.path.join(root_dir,id_dir[0]))

# create transform for data augmentation: normalize, and convert to tensor
mean = np.array([0.5, 0.5, 0.5])
std = np.array([0.5, 0.5, 0.5])
data_transforms = {
    label_dir[0]: transforms.Compose([
        transforms.RandomHorizontalFlip(), #add randomness to data augmentation
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    label_dir[1]: transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}
# define hyperparameters
batch_size = 25
num_epochs = 25

#show image
def imshow(inp, title):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    plt.title(title)
    plt.show()

In [6]:
data = pd.DataFrame(columns=['id','label','image_name'])
data['id'] = 0
data['label'] = 1
data['image_name'] = 2

print(data)

Empty DataFrame
Columns: [id, label, image_name]
Index: []


In [6]:
#create dataset class for breast cancer data

class breastcancerDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        # get root directory
        self.root_dir = root_dir
        # get transform
        self.transform = transform
        # get list of label from directory under root_dir
        id_dir = os.listdir(root_dir)
        label_dir = os.listdir(os.path.join(root_dir,id_dir[0]))
        self.data = pd.DataFrame(columns=['id','label','image_name'])
        tempdata = pd.DataFrame(columns=['id','label','image_name'])
        # loop through id in id_dir and label in label_dir to get image name and label
        for id in id_dir:
            for label in label_dir:
                tempdata['id'] = id
                tempdata['image_name'] = os.listdir(os.path.join(root_dir, id, label))
                tempdata['label'] = label
                self.data = pd.concat([self.data, tempdata],ignore_index = True, axis = 0)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # get image name and label from dataframe
        id = self.data.iloc[idx,0]
        img_name = os.path.join(self.root_dir,self.data.iloc[idx, 0],self.data.iloc[idx, 1],self.data.iloc[idx, 2])
        image = io.imread(img_name)
        label = torch.tensor(self.data.iloc[idx, 1])
                
        if self.transform:
            image = self.transform(image)
            
        return id,image,label

['0', '1']


In [ ]:
dataset = breastcancerDataset(root_dir)
